In [1]:
import sys
sys.path.append("../scripts")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from clock_utils import make_groups

%load_ext autoreload
%autoreload 2

basepath1 = "../data/paper_data/"
basepath2 = "../data/paper_addon_atac/"

In [2]:
# Get preprocessed data
data1 = pd.read_csv(basepath1 + "data_atac_tpm.tsv", index_col=0, sep="\t")
data2 = pd.read_csv(basepath2 + "atac_tpm_ucar.tsv", index_col=0, sep="\t")
data = pd.concat([data1, data2])
data = data.apply(np.log1p)

# # Get meta
meta1 = pd.read_csv(basepath1 + "meta_final.tsv", sep="\t")
meta1 = meta1.loc[meta1.PassesQC_atac, :]
meta2 = pd.read_csv(basepath2 + "meta_ucar.tsv", sep="\t")
meta = pd.concat([meta1[["Subject", "Age"]], meta2[["Subject", "Age"]]])
meta = meta.set_index("Subject", drop=False)

# Match meta and data
data = data.loc[meta.index, :]
labels = meta.Age

groups = make_groups(11, labels)

In [3]:
from sklearn.linear_model import ElasticNet
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

pipes = list()

# Model 1

pipe = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("regressor", ElasticNet(max_iter=5000, tol=0.0005))])

param_grid = dict()
param_grid["regressor__alpha"] = np.logspace(-4, 1, 30) #40
param_grid["regressor__l1_ratio"] = [0.7, 0.8, 0.9, 0.95, 0.99, 1]

In [4]:
from sklearn.model_selection import LeavePGroupsOut

outer_groups_out = 1
scoring = "neg_median_absolute_error"
cv_outer = LeavePGroupsOut(outer_groups_out)
outer_split = list(cv_outer.split(data, labels, groups))

In [5]:
import shelve, os, dbm.dumb
from datetime import datetime

date = datetime.now().strftime("%Y-%m-%d_%H-%M")
outpath = "../clocks/parallel/" + date + "_tpm_all_samples"
os.makedirs(outpath, exist_ok=True)

vars_to_save = ["data", "labels", "groups", "pipe", "param_grid", "outer_split", "scoring"]

dumbdb = dbm.dumb.open(outpath + "/dataset")
dataset = shelve.Shelf(dumbdb)
for key in vars_to_save:
    try:
        dataset[key] = globals()[key]
        print('Shelved {0}'.format(key))
    except TypeError:
        #
        # __builtins__, my_shelf, and imported modules can not be shelved.
        #
        print('ERROR shelving: {0}'.format(key))
dataset.close()
dumbdb.close()

Shelved data
Shelved labels
Shelved groups
Shelved pipe
Shelved param_grid
Shelved outer_split
Shelved scoring
